| 順序 | モジュール名 (Name) | パス (Path) | 機能・実装理由 (Description) |
| --- | --- | --- | --- |
| 1 | configs | `timesfm.configs` | 【基盤】設定定義

<br>モデル設定（`ForecastConfig`等）を定義します。後続のほぼ全てのモジュールで型定義として参照されるため、最初に実装が必要です。 |
| 2 | util | `timesfm.torch.util`<br>

<br>`timesfm.flax.util` | 【基盤】ユーティリティ<br>

<br>計算補助関数、キャッシュ機構（`DecodeCache`）など、レイヤー実装に必要な基本的ツール群です。 |
| 3 | normalization | `timesfm.torch.normalization`<br>

<br>`timesfm.flax.normalization` | 【部品】正規化層<br>

<br>`RMSNorm` などの基本的な正規化レイヤーです。依存関係が少なく、早期に実装可能です。 |
| 4 | dense | `timesfm.torch.dense`<br>

<br>`timesfm.flax.dense` | 【部品】全結合層<br>

<br>残差ブロック（`ResidualBlock`）やフーリエ特徴量埋め込みなど、Transformerの構成要素となります。 |
| 5 | transformer | `timesfm.torch.transformer`<br>

<br>`timesfm.flax.transformer` | 【部品】Transformer層<br>

<br>Attention機構やTransformerブロック本体です。これまでの `dense`, `normalization`, `util` を組み合わせて構築します。 |
| 6 | xreg_lib | `timesfm.utils.xreg_lib` | 【機能】共変量ライブラリ<br>

<br>外部共変量（外生変数）を処理するための独立したロジックです。モデル本体の推論ロジックで使用されます。 |
| 7 | timesfm_2p5_base | `timesfm.timesfm_2p5.timesfm_2p5_base` | 【骨格】モデル基底クラス<br>

<br>モデルの共通インターフェース、前処理、推論のワークフローを定義する抽象クラスです。 |
| 8 | timesfm_2p5_torch<br>

<br>(or _flax) | `timesfm.timesfm_2p5.timesfm_2p5_torch`<br>

<br>`timesfm.timesfm_2p5.timesfm_2p5_flax` | 【統合】モデル実装<br>

<br>これまでに作成した部品（config, layers, base）を統合し、具体的な `TimesFM` モデル（ロード、推論処理）を完成させます。 |
| 9 | timesfm | `timesfm` | 【I/F】APIエンドポイント<br>

<br>ユーザーがライブラリをインポートして利用するためのトップレベルAPIです。 |

# timesfm.configs


## 予測設定 (ForecastConfig) の定義
推論時の最大コンテキスト長、予測期間（ホライズン）、バッチサイズなどの詳細設定を行います。

In [1]:
from timesfm.configs import ForecastConfig

# 予測設定の作成
# TODO: 用途に合わせて各パラメータを調整してください
obj = ForecastConfig(
    max_context=512,                 # モデルに入力する最大過去データ点数 (例: 512)
    max_horizon=96,                  # 一度に予測する最大ステップ数 (例: 96)
    normalize_inputs=True,           # 入力データを正規化するか (推奨: True)
    window_size=None,                # 分解予測時のウィンドウサイズ (通常は None)
    per_core_batch_size=32,          # コアごとのバッチサイズ
    use_continuous_quantile_head=False, # 連続分位点ヘッドを使用するか
    force_flip_invariance=False,     # 反転不変性 (符号反転への対応) を強制するか
    infer_is_positive=False,         # 出力が非負であることを保証するか
    fix_quantile_crossing=True,      # 分位点の交差を修正するか
    return_backcast=False,           # 過去データの再構成 (backcast) を返すか
)

print(f"ForecastConfig created: context={obj.max_context}, horizon={obj.max_horizon}")

ForecastConfig created: context=512, horizon=96


## ランダムフーリエ特徴量設定 (RandomFourierFeaturesConfig) の定義
時系列データの周波数成分を捉えるためのランダムフーリエ特徴量レイヤーの設定を行います。

In [2]:
from timesfm.configs import RandomFourierFeaturesConfig

# ランダムフーリエ特徴量レイヤーの設定作成
# TODO: モデルのアーキテクチャに合わせて値を調整してください
obj = RandomFourierFeaturesConfig(
    input_dims=64,           # 入力次元数 (int)
    output_dims=64,          # 出力次元数 (int)
    projection_stddev=0.01,  # 投影重みの初期化標準偏差 (float)
    use_bias=False,          # バイアス項を使用するか (bool)
)

print(f"RFF Config created: input={obj.input_dims}, output={obj.output_dims}")

RFF Config created: input=64, output=64


## 残差ブロック設定 (ResidualBlockConfig) の定義
モデル内の各層で使用される残差ブロック（Residual Block）の次元数や活性化関数などの構成を定義します。

In [3]:
from timesfm.configs import ResidualBlockConfig

# 残差ブロックの設定作成
# TODO: モデルの各層の設計に合わせて値を調整してください
obj = ResidualBlockConfig(
    input_dims=64,           # 入力次元数 (int)
    hidden_dims=128,         # 隠れ層の次元数 (int)
    output_dims=64,          # 出力次元数 (int)
    use_bias=True,           # バイアス項を使用するか (bool)
    activation="swish",      # 活性化関数 (Literal["relu", "swish", "none"])
)

print(f"ResidualBlock Config created: {obj.input_dims} -> {obj.hidden_dims} -> {obj.output_dims} (act={obj.activation})")

ResidualBlock Config created: 64 -> 128 -> 64 (act=swish)


## 積み上げ型Transformer設定 (StackedTransformersConfig) の定義
モデルの核となるTransformerブロックを何層積み上げるか、および各層の詳細な構成（TransformerConfig）を定義します。

In [4]:
from timesfm.configs import TransformerConfig, StackedTransformersConfig

# 1. 個別のTransformer層の詳細設定
# TODO: モデルの規模（200M等）に合わせて調整してください
transformer_config = TransformerConfig(
    model_dims=1024,
    hidden_dims=4096,
    num_heads=16,
    attention_norm="rms",
    feedforward_norm="rms",
    qk_norm="rms",
    use_bias=False,
    use_rotary_position_embeddings=True,
    ff_activation="swish",
    fuse_qkv=True
)

# 2. Transformerを積み上げる設定の作成
# TODO: レイヤー数を指定してください
obj = StackedTransformersConfig(
    num_layers=20,            # 積み上げるTransformerの層数 (int)
    transformer=transformer_config  # 上記で定義したTransformerConfigオブジェクト
)

print(f"Stacked Transformers: {obj.num_layers} layers of {obj.transformer.model_dims} dims")

Stacked Transformers: 20 layers of 1024 dims


## Transformer構成設定 (TransformerConfig) の定義

TimesFMモデルの核となるTransformerブロックの内部アーキテクチャを詳細に定義します。この設定は、アテンション機構の挙動、正規化の手法、およびフィードフォワードネットワークの構成を決定します。

### 主な設定項目:
* **次元数設定**: モデルの基底次元 (`model_dims`) とフィードフォワード層の隠れ次元 (`hidden_dims`) を指定します。
* **正規化 (Normalization)**: `attention_norm` や `feedforward_norm` に `rms` (Root Mean Square Layer Normalization) を指定し、学習の安定化を図ります。
* **Q/K 正規化 (QK Norm)**: クエリ(Q)とキー(K)に対して正規化を行うことで、アテンションスコアの極端な増大を抑制します。
* **埋め込み方式**: `use_rotary_position_embeddings` (RoPE) を有効にし、相対的な位置情報を効率的に扱います。
* **効率化**: `fuse_qkv` を True にすることで、Q, K, V の計算を 1 つの行列演算に統合し、計算速度を向上させます。

In [5]:
from timesfm.configs import TransformerConfig

# Transformerの詳細なアーキテクチャ設定を作成
# TODO: 構築するモデルのパラメータ数や計算リソースに応じて値を調整してください
obj = TransformerConfig(
    model_dims=1024,                      # モデルの基底次元数 (int)
    hidden_dims=4096,                     # フィードフォワード層の隠れ次元数 (int)
    num_heads=16,                         # マルチヘッドアテンションのヘッド数 (int)
    attention_norm="rms",                 # アテンション層の正規化手法 (Literal["rms"])
    feedforward_norm="rms",               # フィードフォワード層の正規化手法 (Literal["rms"])
    qk_norm="rms",                        # Query/Keyへの正規化適用 (Literal["rms", "none"])
    use_bias=False,                       # 線形層でバイアス項を使用するか (bool)
    use_rotary_position_embeddings=True,  # Rotary Positional Embeddingsを使用するか (bool)
    ff_activation="swish",                # フィードフォワード層の活性化関数 (Literal["relu", "swish", "none"])
    fuse_qkv=True                         # Q, K, Vの計算を統合するか (bool)
)

print(f"Transformer Config created: {obj.model_dims} dims, {obj.num_heads} heads, act={obj.ff_activation}")

Transformer Config created: 1024 dims, 16 heads, act=swish


# timesfm.flax.dense


## デコードキャッシュ設定 (DecodeCache) の定義

TimesFM 2.5 の Flax 実装において、自己回帰的（Autoregressive）なデコードを効率化するためのキャッシュ機構を定義します。このキャッシュは、トランスフォーマーのアテンション計算における過去の Key と Value を保持し、推論速度を大幅に向上させます。

### 構成要素の詳細:
* **next_index**: 次に書き込むキャッシュのインデックスを保持する配列です。バッチごとに管理されます。
* **num_masked**: マスクされているトークンの数を管理します。
* **key / value**: アテンション機構で再利用される Key と Value のテンソルです。形状は `[batch, length, heads, dims]` となります。
* **イミュータブルな設計**: このクラスは `dataclasses.dataclass` かつ `frozen=True` で定義されており、関数型プログラミングのパラダイムに沿った安全な状態管理を可能にします。

In [8]:
import jax.numpy as jnp
from timesfm.flax.util import DecodeCache

# 1. キャッシュの次元定義 (例: batch_size=32, seq_len=512, num_heads=16, head_dim=64)
batch_size = 32
max_len = 512
num_heads = 16
head_dim = 64

# 2. デコードキャッシュのインスタンス化
# TODO: モデルの推論状態に合わせて各配列の値を初期化してください
obj = DecodeCache(
    next_index=jnp.zeros((batch_size,), dtype=jnp.int32),  # 次の書き込み位置
    num_masked=jnp.zeros((batch_size,), dtype=jnp.int32),  # マスクされたトークン数
    key=jnp.zeros((batch_size, max_len, num_heads, head_dim), dtype=jnp.float32),   # Keyキャッシュ
    value=jnp.zeros((batch_size, max_len, num_heads, head_dim), dtype=jnp.float32)  # Valueキャッシュ
)

print(f"Flax DecodeCache initialized: Key shape {obj.key.shape}")

Flax DecodeCache initialized: Key shape (32, 512, 16, 64)


## RandomFourierFeatures（ランダムフーリエ特徴量）で入力を周期特徴へ写像する

`RandomFourierFeatures` は、入力 `x`（例：時刻特徴や連続値特徴）を、ランダムな線形射影→ `sin` / `cos` 変換で高次元の周期特徴へ変換する層です（Random Fourier Features：カーネル近似の定番テクニック）。:contentReference[oaicite:0]{index=0}

### 典型的な引数（実装差があるので実際は signature を確認）
- `scale`（スケール）：周波数の大きさ。大きいほど細かく振動する特徴になる
- `n_features`（特徴数）：出力の次元。`sin` と `cos` を連結する都合で偶数になりがち

### 入力テンソル形状
- `x: Float[Array, 'b ... i']`
  - `b`：バッチ次元
  - `...`：任意の追加次元（例：系列長）
  - `i`：入力特徴次元（最後の次元が入力チャンネル）

※ Flax(linen) の `nn.Module` なら、単体テストは `init(...)` で変数を作ってから `apply(...)` で推論します（`__call__` を直接は叩かない）。


In [11]:
import inspect
import jax
import jax.numpy as jnp
from timesfm.flax.dense import RandomFourierFeatures

def build_rff(**preferred_kwargs):
    """
    timesfm の RandomFourierFeatures はバージョン差で引数名が変わる可能性があるので、
    signature に存在する引数だけを渡す。
    """
    sig = inspect.signature(RandomFourierFeatures)
    allowed = set(sig.parameters.keys())
    filtered = {k: v for k, v in preferred_kwargs.items() if k in allowed}
    return RandomFourierFeatures(**filtered)

# --- ここが「TODO: __init__ args」に相当 ---
# よくある名前：scale / n_features（実際に存在するものだけ渡されます）
obj = build_rff(scale=1.0, n_features=256)

# --- ここが「TODO: x」に相当 ---
# x: Float[Array, 'b ... i'] 例）(batch=2, length=8, input_dim=3)
x = jnp.ones((2, 8, 3), dtype=jnp.float32)

# Flax Module の一般的な単体テスト流儀：init → apply
# （__call__ を単体で直接叩かない）
rng = jax.random.PRNGKey(0)

if hasattr(obj, "init") and hasattr(obj, "apply"):
    variables = obj.init(rng, x)
    result = obj.apply(variables, x)
else:
    # もし Flax Module ではなく単なる callable 実装だった場合のフォールバック
    result = obj(x)

print("input shape :", x.shape, x.dtype)
print("output shape:", result.shape, result.dtype)


ModuleNotFoundError: No module named 'flax'